<a href="https://colab.research.google.com/github/pitfisher/miem_face_recognition/blob/main/FR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install deepface
!pip install matplotlib

In [7]:
import cv2
import time
import os
import glob
from PIL import Image, ImageOps, ImageDraw
from deepface import DeepFace
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

In [8]:
input_folder = '/tf/data/upd_face_dataset'

resolutions = [1024, 768, 512, 256, 224, 128, 112, 96, 64, 32]

In [9]:
model_names = [
    "VGG-Face",
    "Facenet",
    "Facenet512",
    "OpenFace",
    "DeepFace",
    "DeepID",
    "ArcFace",
    "SFace"
]

In [10]:
thresholds = {
     "VGG-Face": 1.17,
    "Facenet": 0.8,
    "Facenet512": 1.04,
    "OpenFace": 0.55,
    "DeepFace": 0.64,
    "DeepID": 0.17,
    "ArcFace": 1.13,
    "SFace": 1.055,
}

In [207]:
def crop_to_square(image, face_info, border_pixels=0):
    # print(f"Face info: {face_info}")
    x, y, w, h = face_info['facial_area']['x'], face_info['facial_area']['y'], face_info['facial_area']['w'], face_info['facial_area']['h']

    # print(f"x: {x}, y: {y}, w: {w}, h: {h}")
    square_size = max(w, h) + 2 * border_pixels
    # new_x = max(0, x + w // 2 - square_size // 2)
    # new_y = max(0, y + h // 2 - square_size // 2)

    new_x = max(0, x)
    new_y = max(0, y)

    img = Image.fromarray(image)
    cropped_img = img.crop((new_x - border_pixels, new_y - border_pixels, new_x + square_size, new_y + square_size))

    return cropped_img

In [11]:
def crop_to_square_pil(image, face_info, border_pixels=0):
    # print(f"Face info: {face_info}")
    x, y, w, h = face_info['facial_area']['x'], face_info['facial_area']['y'], face_info['facial_area']['w'], face_info['facial_area']['h']

    # print(f"x: {x}, y: {y}, w: {w}, h: {h}")
    square_size = max(w, h) + 2 * border_pixels
    # new_x = max(0, x + w // 2 - square_size // 2)
    # new_y = max(0, y + h // 2 - square_size // 2)

    new_x = max(0, x)
    new_y = max(0, y)
    cropped_image = image.crop((new_x - (h-w)//2 - border_pixels, new_y - border_pixels, new_x - (h-w)//2 + square_size, new_y + square_size))

    return cropped_image

In [208]:
def resize_image(image, new_resolution):

    resized_img = image.resize(new_resolution, Image.LANCZOS)
    return resized_img

In [12]:
def resize_image_pil(image, new_resolution):
    resized_img = image.resize(new_resolution, Image.LANCZOS)
    return resized_img

In [13]:
def save_to_csv(data, columns, filename):
    df = pd.DataFrame(data, columns=columns)
    df.to_csv(filename + '.csv', index=False)

In [ ]:
for model_name in model_names:
   df_list = []
   for resolution in resolutions:
      
       files = glob.glob(os.path.join(folder_path, "**", "*.JPG"), recursive=True)
    
       for img_path in tqdm(files, desc=f'Creating images cropped&resized to {resolution} for {model_name}', unit='image'):
        rel_path = os.path.relpath(img_path, folder_path)
        rel_path = rel_path.replace(os.path.sep, "_")
        output_path = os.path.join(tmp_folder, f"{rel_path[:-4]}_{resolution}.jpg")
      
        if not os.path.exists(output_path):
            image = Image.open(img_path)
            image = ImageOps.exif_transpose(image)
            
            faces = DeepFace.extract_faces(img_path=img_path, detector_backend="retinaface")
            for face_info in faces:
                cropped_face = crop_to_square_pil(image, face_info)
                resized_img = resize_image_pil(cropped_face, (resolution, resolution))
                resized_img.save(output_path)

In [ ]:
import ipyplot


files = glob.glob(os.path.join(input_folder, "**", "*.JPG"), recursive=True)

img_path = files[44]
face_info = DeepFace.extract_faces(img_path=img_path, detector_backend="retinaface")[0]
x, y, w, h = face_info['facial_area']['x'], face_info['facial_area']['y'], face_info['facial_area']['w'], face_info['facial_area']['h']

image = Image.open(img_path)
# https://github.com/python-pillow/Pillow/issues/4703#issuecomment-645219973
image = ImageOps.exif_transpose(image)
draw = ImageDraw.Draw(image)

image_array = np.array(image)
image_array = cv2.cvtColor(image_array, cv2.COLOR_RGB2BGR)
display(image)

cropped_img = crop_to_square(image_array, face_info)
img1 = np.array(cropped_img)

metrics = []
plt.figure()

#subplot(r,c) provide the no. of rows and columns
f, axarr = plt.subplots(1,2) 

for resolution in resolutions:
    resized_img = resize_image(cropped_img, (resolution, resolution))
    img2 = np.array(resized_img)
    axarr[0].imshow(img1)
    axarr[1].imshow(img2)
    plt.show()
    result = DeepFace.verify(img1_path=img1,
                             img2_path=img2,
                             enforce_detection=False,
                             detector_backend="retinaface",
                             model_name=model_name,
                             distance_metric="euclidean_l2"
                             )
    print(f'distance:{result["distance"]:.5f}')


In [ ]:
files = glob.glob(os.path.join(input_folder, "**", "*.JPG"), recursive=True)

data = {}

for model_name in model_names:

    for img_path in tqdm(files, desc=f"Processing images with {model_name}", unit="image"):

        path = img_path.split(os.path.sep)
        filename = f'{path[-2]}_{path[-1]}'
        ds_name = f'{model_name}_{path[-4]}_{path[-3]}'

        try:
            face_info = DeepFace.extract_faces(img_path=img_path, detector_backend="retinaface")[0]
        except ValueError as e:
            print(f"Face detection failed {img_path}: {e}")
            continue

        image = Image.open(img_path)
        image = ImageOps.exif_transpose(image)
        image_array = np.array(image)
        
        cropped_img = crop_to_square_pil(image_array, face_info)
        img1 = np.array(cropped_img)

        metrics = []
        # plt.figure()
        
        #subplot(r,c) provide the no. of rows and columns
        # f, axarr = plt.subplots(1,2) 
 
        for resolution in resolutions:

            resized_img = resize_image_pil(cropped_img, (resolution, resolution))
            img2 = np.array(resized_img)
            # axarr[0].imshow(img1)
            # axarr[1].imshow(img2)
            # plt.show()
            start_time = time.perf_counter()

            try:
                result = DeepFace.verify(img1_path=img1,
                                         img2_path=img2,
                                         enforce_detection=False,
                                         detector_backend="retinaface",
                                         model_name=model_name,
                                         distance_metric="euclidean_l2"
                                         )
                end_time = time.perf_counter()
                fin_time = end_time - start_time

                metrics.append(f'{result["distance"]:.5f}')
                metrics.append(f'{fin_time:.5f}')

            except ValueError as e:
                print(f"Verification failed {img_path}: {e}")
                metrics.extend([None, None])

        if ds_name not in data:
            data[ds_name] = []

        data[ds_name].append([filename] + metrics)

columns = ['name'] + [f"{res}_{metric}" for res in resolutions for metric in ["distance", "time"]]

for ds_name, ds in data.items():
    save_to_csv(ds, columns, ds_name)